In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('Datasetv2.csv')  

In [ ]:
# df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
import nltk

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

In [ ]:
orig_lines = df['Trip Name'].value_counts().index.to_numpy()

In [ ]:
new_lines = []
for line in orig_lines:
    s = ''.join([i for i in line if not i.isdigit()])
    new_lines.append(s)

In [ ]:
import re
new_new_lines = []

for line in new_lines:
    result = re.sub("[\(\[].*?[\)\]]", "", line)
    new_new_lines.append(result)

In [ ]:
lines = [line.replace('TUI', '').replace('Premium', '').replace('SL', '').replace('Prestige', '').replace('.', ' ').replace('-', ' ').replace('+', ' ').replace('*', ' ').replace('RU', '').replace('MOW', '').replace('GDS', '') for line in new_new_lines]

In [ ]:
tour_name_map_dict = {}
for orig_line, line in zip(orig_lines, lines):
    sentences = nltk.sent_tokenize(line)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

    entities = []
    for tree in chunked_sentences:
        entities.extend(extract_entity_names(tree))
    if len(entities) > 0:
        if 0 < len(entities[0].split()) < 3:
            print(orig_line, '#', entities[0].split()[-1])
            tour_name_map_dict[orig_line] = entities[0].split()[-1]
        elif len(entities[0].split()) >= 3:
            print(orig_line, '##', entities)
            tour_name_map_dict[orig_line] = entities[0].split()[1]

In [ ]:
tour_name_map_dict['Czech Republic Prague MOW GDS'] = 'Prague'
tour_name_map_dict['UAE W17/18 VKO'] = 'Dubai'

In [ ]:
df['Trip Name'] = df['Trip Name'].map(tour_name_map_dict)

In [ ]:
df['Trip Name'].value_counts()

In [ ]:
# df['Age'] = df['Age'].astype(int)
# df['Gender'] = df['Gender'].astype(str)
# df['Trip Type'] = df['Trip Type'].astype(str)
# df['Order Type'] = df['Order Type'].astype(str)
# df['Country'] = df['Country'].astype(str)
# df['Region'] = df['Region'].astype(str)
# df['Star Rate'] = df['Star Rate'].astype(str)
# df['Group Type'] = df['Group Type'].astype(str)
# df['Trip Name'] = df['Trip Name'].astype(str)
# df['Meal Option'] = df['Meal Option'].astype(str)
# df['Destination City'] = df['Destination City'].astype(str)
# df['Departure City'] = df['Departure City'].astype(str)

In [ ]:
# col_names = {'Пол': 'Gender', 'День рождения': 'Birthday', 
#             'Возраст клиента': 'Age', 'Заявка № ': 'Order', 
#             'Дата бронирования': 'Order Date', 'Дата начала тура': 'Trip Start Date',
#             'Год': 'Year', 'Ночей': 'N Nights', 'Вид тура': 'Trip Type', 
#             'Тип заявки': 'Order Type', 'Страна тура': 'Country', 
#             'Регион отеля': 'Region', 'Туристов': 'N Tourists',
#             'Туристы, взрослые': 'Adult Tourists', 'Туристы, дети': 'Kid Tourists',
#             'Младенцы': 'Baby Tourists', 'Звездность': 'Star Rate', 
#             'Состав группы (Заявка) (Заявка)': 'Group Type', 'Сумма в $': 'Price', 
#             'Наименование тура': 'Trip Name', 'ИД клиента': 'Client ID', 
#             'Глубина продаж': 'Booking Depth', 'Тип питания': 'Meal Option', 
#             'Город Отеля': 'Destination City', 'Город отправления': 'Departure City'}
# df = df.rename(columns=col_names)

In [ ]:
df['Birthday'] = pd.to_datetime(df['Birthday'])
df.loc[df['Birthday'].notnull(), 'Birthday'] = df.loc[df['Birthday'].notnull(), 'Birthday'].apply(lambda x: x.timestamp()) 

df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Order Date'] = df['Order Date'].apply(lambda x: x.timestamp()) 

df['Trip Start Date'] = pd.to_datetime(df['Trip Start Date'])
df['Trip Start Date'] = df['Trip Start Date'].apply(lambda x: x.timestamp()) 


In [ ]:
# df_meal = pd.read_excel('MEALS - code.xlsx')
# df_meal['Unnamed: 0'] = df_meal['Unnamed: 0'].astype(str)
# df_meal.set_index(['Unnamed: 0'], inplace=True)
# meal_map = df_meal['Code meal'].to_dict()
# df['Meal Option'] = df['Meal Option'].map(meal_map)

In [ ]:
# df_stars = pd.read_excel('STARS - code.xlsx')
# df_stars['Unnamed: 0'] = df_stars['Unnamed: 0'].astype(str)
# df_stars.set_index(['Unnamed: 0'], inplace=True)
# stars_map = df_stars['Code'].to_dict()
# df['Star Rate'] = df['Star Rate'].map(stars_map)

In [ ]:
df = df.set_index(['Client ID'])

In [ ]:
thresh_min = 16 # could be smaller e.g. 16
# thresh_max = 60 
df = df.loc[df['Age'] >= thresh_min]
# df = df.loc[df['Age'] <= thresh_max]

In [ ]:
df['Age'] = df['Age'].astype(int)

In [ ]:
def categorize(series):
    tmp_dict = series.value_counts(dropna=False).to_dict()
    tmp_list = sorted(tmp_dict.items(), key=lambda item: -item[1])
    map_dict = {}
    for i, item in enumerate(tmp_list):
        map_dict[item[0]] = i
    cat_series = series.map(map_dict)
    return cat_series, map_dict

In [ ]:
df_new = pd.DataFrame(index=df.index)

In [ ]:
df_new['Gender'], gender_map = categorize(df['Gender'])

In [ ]:
df_new['Year'], year_map = categorize(df['Year'])

In [ ]:
df_new['Trip Type'], trip_type_map = categorize(df['Trip Type'])

In [ ]:
df_new['Order Type'], order_type_map = categorize(df['Order Type'])

In [ ]:
df_new['Country'], country_map = categorize(df['Country'])

In [ ]:
df.loc[df['Region'].isna(), 'Region'] = df.loc[df['Region'].isna()]['Trip Name']

In [ ]:
df_new['Region'], region_map = categorize(df['Region'])

In [ ]:
df_new['Star Rate'], star_map = categorize(df['Star Rate'])

In [ ]:
df_new['Group Type'], group_type_map = categorize(df['Group Type'])

In [ ]:
# df['Trip Name'], trip_name_map = categorize(df['Trip Name'])

In [ ]:
df_new['Meal Option'], meal_opt_map = categorize(df['Meal Option'])

In [ ]:
df_new['Destination City'], dest_city_map = categorize(df['Destination City'])

In [ ]:
df_new['Departure City'], dep_city_map = categorize(df['Departure City'])

In [ ]:
df_new['Age'], age_map = categorize(df['Age'])

In [ ]:
df_new['N Nights'], nights_map = categorize(df['N Nights'])

In [ ]:
df_new['N Tourists'], ntourists_map = categorize(df['N Tourists'])

In [ ]:
df_new['Adult Tourists'], adult_map = categorize(df['Adult Tourists'])

In [ ]:
df_new['Baby Tourists'], baby_map = categorize(df['Baby Tourists'])

In [ ]:
df_new['Kid Tourists'], kid_map = categorize(df['Kid Tourists'])

In [ ]:
df_new

In [ ]:
df_continius = df[['Birthday', 'Order', 'Order Date', 'Trip Start Date', 'Price', 'Booking Depth']]

In [ ]:
dfdf = pd.concat([df_new, df_continius], axis=1)

In [ ]:
dfdf.columns.shape

In [ ]:
df.columns.shape

In [ ]:
dfdf.to_csv('DatasetV4.csv')

In [ ]:
def ohe(df, cols):
    dfs = []
    for col in cols:
        ohe_df = pd.get_dummies(df[col])
        dfs.append(ohe_df)
    return pd.concat(dfs, axis=1)

In [ ]:
dfdfdf = ohe(dfdf, ['Gender', 'Year', 'Trip Type', 'Order Type', 'Country', 'Region',
       'Star Rate', 'Group Type', 'Meal Option', 'Destination City',
       'Departure City', 'Age', 'N Nights', 'N Tourists', 'Adult Tourists',
       'Kid Tourists', 'Baby Tourists'])

In [ ]:
# dfdfdf = df_feat[['Birthday', 'Order Date', 'Trip Start Date', 'Price', 'Booking Depth']]

In [ ]:
dfdfdf

In [ ]:
# from sklearn.manifold import TSNE
from tsnecuda import TSNE
X_embedded = TSNE(n_components=2, perplexity=1024.0, learning_rate=50.0).fit_transform(dfdfdf.to_numpy())
# X_embedded = TSNE(n_components=2).fit_transform()

In [ ]:
X_embedded

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=12, weight_concentration_prior=3).fit_predict(X_embedded)

In [ ]:
bgm

In [ ]:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=24).fit_predict(X_embedded)

In [ ]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=2)
X_cat = pca.fit_transform(dfdfdf.to_numpy())

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dfdfdf_norm = scaler.fit_transform(dfdfdf.to_numpy())
pca = PCA(n_components=2)
X_temporal = pca.fit_transform(dfdfdf_norm)

In [ ]:
# plt.scatter(X_all[:, 0], X_all[:, 1]) 

In [ ]:
plt.scatter(X_cat[:, 0], X_cat[:, 1]) 

In [ ]:
fig = plt.figure(figsize=(20, 20))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=bgm) 